In [2]:
import numpy.linalg as linalg
class Yao():
    def __init__(self, data,  max_lag_to_check):
        self.observations=data
        self.max_lag=max_lag_to_check
        self.n_factor=None
        self.M=None
        self.loading=None
        self.factors=None
    def fit(self):
        def auto_cov(y,k):
            """calculate the auto covariance matrix of vector time series y with its k-lagged version.
            y is an numpy array of dimension p*T"""
            mean=np.array([np.mean(y,axis=1)]).T
            #print(mean)
            T=y.shape[1]
            y_t=y
            y_t_plus_k=np.roll(y_t,-k,axis=1)
            y_t=y_t[:,:T-k]-mean
            y_t_plus_k=y_t_plus_k[:,:T-k]-mean

            return y_t_plus_k@(y_t.T)/(T-k)
        
        def construct_M(y,k_0):
            Sigma=auto_cov(y,1)
            M=Sigma@(Sigma.T)
            for k in range(2,k_0+1):
                Sigma=auto_cov(y,k)
                M=M+Sigma@(Sigma.T)
            return M

        def eigen_decomp(M):
            eigenValues, eigenVectors = linalg.eig(M)

            idx = eigenValues.argsort()[::-1]   
            eigenValues = eigenValues[idx]
            eigenVectors = eigenVectors[:,idx]

            return eigenValues,eigenVectors
        
        def select_r(eigen_Values):
            limit=len(eigen_Values)//2+1
            e_values=eigen_Values[:limit]
            ratios=np.roll(e_values,-1)[:-1]/e_values[:-1]
            print(ratios)
            return np.argmin(ratios)+1
        
        self.M=construct_M(self.observations,self.max_lag)
        
        eigenValues,eigenVectors=eigen_decomp(self.M)
        
        self.n_factor=select_r(eigenValues)
        
        self.loading=eigenVectors[:,:self.n_factor]
        
        self.factors=self.loading.T@self.observations